In [3]:
#Đọc dữ liệuliệu
import pandas as pd
df = pd.read_csv("../../crawl_data/raw_data/cellphones.csv", on_bad_lines='skip')
print(df.shape)

df['Price sale'] = pd.to_numeric(df['Price sale'].str.replace(r'[^\d]', '', regex=True), errors='coerce')
df['Price origin'] = pd.to_numeric(df['Price origin'].str.replace(r'[^\d]', '', regex=True), errors='coerce')

df = df.dropna(subset=['Price sale'])  # Xóa dòng không có giá bán

df = df.rename(columns={
    'Title': 'Tên Sản Phẩm',
    'Information': 'Thông tin',
    'Price sale': 'Giá Bán',
    'Price origin': 'Giá Gốc',
    'Loại card đồ họa': 'GPU',
    'Dung lượng RAM': 'RAM',
    'Loại RAM': 'Loại RAM',
    'Số khe ram': 'Số khe RAM',
    'Ổ cứng': 'Bộ Nhớ',
    'Kích thước màn hình': 'Kích Thước',
    'Công nghệ màn hình': 'Màn Hình',
    'Pin': 'Dung Lượng Pin',
    'Hệ điều hành': 'Hệ Điều Hành',
    'Độ phân giải màn hình': 'Độ Phân Giải',
    'Loại CPU': 'CPU',
    'Cổng giao tiếp': 'Cổng Giao Tiếp',
    'Chip AI': 'Chip AI'
})



(400, 17)


In [37]:
print(df['Tên Sản Phẩm'].unique())

['Laptop HP 250 G9 AG2K7AT' 'Laptop HP Pavilion 15-EG3093TU 8C5L4PA'
 'Laptop ASUS  Vivobook 14 OLED A1405VA-KM257W'
 'Laptop ASUS TUF Gaming A15 FA506NCR-HN047W'
 'Laptop Lenovo LOQ 15IRX9 83DV012LVN'
 'Laptop Acer Aspire 3 A314-42P-R3B3 NX.KSFSV.001'
 'Laptop MSI Katana 15 B13UDXK-2270VN V2'
 'Laptop Acer Aspire Lite 15 AL15-71P-517D'
 'Laptop Lenovo IdeaPad Slim 3 16IRH10 83K20002VN'
 'MacBook Air M4 13 inch 2025 10CPU 8GPU 16GB 256GB | Chính hãng Apple Việt Nam'
 'Apple MacBook Air M2 2024 8CPU 8GPU 16GB 256GB I Chính hãng Apple Việt Nam'
 'Laptop Lenovo LOQ 15ARP9 83JC007HVN'
 'Laptop Gaming Acer Nitro V ANV15-51-58AN'
 'Laptop ASUS Gaming VivoBook K3605ZC-RP564W'
 'MacBook Pro 14 M4 10CPU 10GPU 16GB 512GB | Chính hãng Apple Việt Nam'
 'Laptop Acer Gaming Aspire 7 A715-76-53PJ'
 'Laptop ASUS TUF Gaming F15 FX507ZC4-HN095W'
 'Mac mini M4 2024 10CPU 10GPU 16GB 256GB | Chính hãng Apple Việt Nam'
 'Laptop Gaming Acer Nitro V ANV15-51-72VS NH.QNASV.004'
 'Laptop Acer Gaming Aspire 5 A5

In [38]:
#Thêm cột Độ Dày
import re

# Danh sách ánh xạ từ khóa nhận diện trong tên -> độ dày (mm)
mapping = {
    r'macbook air': 11.3,
    r'macbook pro': 15.5,
    r'mac mini': 36.0,
    r'mac studio': 95.0,
    r'imac': 11.5,
    r'hp pavilion': 17.9,
    r'hp victus|hp gaming': 23.5,
    r'hp envy': 16.9,
    r'hp 240|hp 250': 19.9,
    r'hp 15|hp 14': 18.0,
    r'asus tuf': 24.5,
    r'asus vivobook s': 14.9,
    r'asus vivobook': 17.7,
    r'asus zenbook': 13.9,
    r'asus expertbook': 18.0,
    r'asus rog zephyrus g16': 22.8,
    r'asus rog zephyrus': 25.9,
    r'asus rog': 24.0,
    r'lenovo loq': 22.7,
    r'ideapad slim': 19.5,
    r'ideapad': 19.5,
    r'legion slim': 17.1,
    r'legion 5 pro': 25.9,
    r'legion 5': 24.5,
    r'yoga duet': 9.2,
    r'yoga slim 9': 13.9,
    r'yoga slim 7': 14.9,
    r'yoga slim': 14.9,
    r'yoga pro': 15.5,
    r'thinkpad e14': 19.9,
    r'thinkpad e16': 20.4,
    r'thinkbook 14': 17.1,
    r'thinkbook 16': 17.1,
    r'v14': 19.2,
    r'v15': 19.5,
    r'latitude': 17.9,
    r'prestige': 16.0,
    r'cyborg': 22.5,
    r'msi modern': 18.0,
    r'msi thin': 22.8,
    r'msi gaming': 25.0,
    r'msi katana': 24.9,
    r'aspire 7': 22.9,
    r'aspire 5': 18.8,
    r'aspire 3': 19.5,
    r'aspire lite': 17.9,
    r'nitro': 25.8,
    r'gigabyte g5|gigabyte g6': 23.4,
    r'gigabyte mf': 23.4,
    r'dell inspiron': 18.0,
    r'dell vostro': 18.0,
    r'dell latitude': 17.9,
    r'lg gram': 11.5,
    r'masstel e140': 13.9,
    r'swift go': 15.0,
    r'vaio fe': 17.2,
    r'vivobook 14 oled a1405va': 17.7,
    r'vivobook k3605zc': 24.0,
    r'vivobook k3605zf': 24.0,
    r'zephyrus g14 ga403uu': 25.9,
    r'zephyrus g16 gu603vu': 22.8,
    r'matebook d15': 16.9,
    r'swift lite': 15.0,
    r'dell xps 13': 17.2,
    r'slim 3 14irh10': 19.5,
    r'vivobook k3605zu': 24.0,
    r'vivobook s 16 oled s5606ma': 17.9
}


# Hàm tìm độ dày dựa vào từ khóa trong tên laptop
def gan_do_day_tu_mapping(ten_laptop):
    ten = ten_laptop.lower()
    for pattern, do_day in mapping.items():
        if re.search(pattern, ten):
            return do_day
    return ten_laptop 

# Thêm cột độ dày bằng cách apply hàm
df['Độ Dày'] = df['Tên Sản Phẩm'].apply(gan_do_day_tu_mapping)
print(df['Độ Dày'].unique())

[19.9 17.9 17.7 24.5 22.7 19.5 24.9 11.3 24.  15.5 22.9 36.  18.8 23.5
 25.8 18.  13.9 11.5 23.4 25.  25.9  9.2 16.9 17.1 20.4 14.9 19.2 95.
 22.5 16.  22.8 15.  17.2]


In [ ]:
#Thêm cột trọng lượng
import re
mapping = {
    r'macbook air': 1.24,
    r'macbook pro': 1.56,
    r'mac mini': 1.18,
    r'mac studio': 2.7,
    r'imac': 4.5,
    r'hp pavilion': 1.75,
    r'hp victus|hp gaming': 2.3,
    r'hp envy': 1.6,
    r'hp 240|hp 250': 1.52,
    r'hp 15|hp 14': 1.6,
    r'asus tuf': 2.3,
    r'asus vivobook s': 1.6,
    r'asus vivobook': 1.7,
    r'asus zenbook': 1.4,
    r'asus expertbook': 1.5,
    r'asus rog zephyrus g16': 2.1,
    r'asus rog zephyrus': 1.8,
    r'asus rog': 2.3,
    r'lenovo loq': 2.4,
    r'ideapad slim': 1.5,
    r'ideapad': 1.6,
    r'legion slim': 2.0,
    r'legion 5 pro': 2.5,
    r'legion 5': 2.4,
    r'yoga duet': 0.8,
    r'yoga slim 9': 1.3,
    r'yoga slim 7': 1.4,
    r'yoga slim': 1.4,
    r'yoga pro': 1.5,
    r'thinkpad e14': 1.6,
    r'thinkpad e16': 1.8,
    r'thinkbook 14': 1.6,
    r'thinkbook 16': 1.8,
    r'v14': 1.5,
    r'v15': 1.7,
    r'latitude': 1.7,
    r'prestige': 1.6,
    r'cyborg': 2.0,
    r'msi modern': 1.7,
    r'msi thin': 2.1,
    r'msi gaming': 2.3,
    r'msi katana': 2.3,
    r'aspire 7': 2.2,
    r'aspire 5': 1.8,
    r'aspire 3': 1.7,
    r'aspire lite': 1.6,
    r'nitro': 2.5,
    r'gigabyte g5|gigabyte g6': 2.3,
    r'gigabyte mf': 2.3,
    r'dell inspiron': 1.8,
    r'dell vostro': 1.7,
    r'dell latitude': 1.7,
    r'lg gram': 1.2,
    r'masstel e140': 1.3,
    r'swift go': 1.3,
    r'vaio fe': 1.5,
    r'vivobook 14 oled a1405va': 1.4,
    r'vivobook k3605zc': 2.1,
    r'vivobook k3605zf': 2.1,
    r'zephyrus g14 ga403uu': 1.8,
    r'zephyrus g16 gu603vu': 2.1,
    r'matebook d15': 1.6,
    r'swift lite': 1.3,
    r'dell xps 13': 1.2,
    r'slim 3 14irh10': 1.5,
    r'vivobook k3605zu': 2.1,
    r'vivobook s 16 oled s5606ma': 1.9
}
# Hàm tìm trọng lượng dựa vào từ khóa trong tên laptop
def gan_trong_luong_tu_mapping(ten_laptop):
    ten = ten_laptop.lower()
    for pattern, trong_luong in mapping.items():
        if re.search(pattern, ten):
            return trong_luong
    return ten_laptop 

# Thêm cột độ dày bằng cách apply hàm
df['Trọng Lượng'] = df['Tên Sản Phẩm'].apply(gan_do_day_tu_mapping)
print(df['Trọng Lượng'].unique())

[1.52 1.75 1.4  2.3  2.4  1.7  1.6  1.5  1.24 2.1  1.56 2.2  1.18 1.8
 2.5  1.3  1.2  4.5  0.8  1.9  2.7  2.  ]


In [40]:
#Thêm Hãng Sản Xuất
import pandas as pd
import re

def extract_brand(df, column='Tên Sản Phẩm'):
    brand_mapping = {
        r'\bapple\b': 'Apple',
        r'\basus\b': 'ASUS',
        r'\blenovo\b': 'Lenovo',
        r'\bhp\b': 'HP',
        r'\bdell\b': 'Dell',
        r'\bac(?:er)?\b': 'Acer',
        r'\bmsi\b': 'MSI',
        r'\bvaio\b': 'Vaio',
        r'\blg\b': 'LG',
        r'\bhuawei\b': 'Huawei',
        r'\bgigabyte\b': 'Gigabyte',
        r'\bmasstel\b': 'Masstel',
        r'\bchuwi\b': 'Chuwi',
        r'\bava\b': 'AVA',
        r'\bxiaomi\b': 'Xiaomi',
        r'\bmouse\b': 'Mouse',
        r'\bsamsung\b': 'Samsung',
        r'\bsurface\b|\bmicrosoft\b': 'Microsoft',
    }

    def map_brand(name):
        name = name.lower()
        for pattern, brand in brand_mapping.items():
            if re.search(pattern, name):
                return brand
        return 'Khác'  # Trường hợp không khớp hãng nào

    df['Hãng Sản Xuất'] = df[column].apply(map_brand)
    return df
df = extract_brand(df)

print(df['Hãng Sản Xuất'].unique())


['HP' 'ASUS' 'Lenovo' 'Acer' 'MSI' 'Apple' 'Dell' 'Masstel' 'LG'
 'Gigabyte' 'Huawei' 'Vaio']


In [41]:
# Xuất dữ liệu đã làm sạch
df.to_csv("../clean_data_train/clean_cellphones.csv", index=False)